In [20]:
import tensorflow as tf
from numpy import pi
from keras.backend import sigmoid
from keras.models import Sequential, load_model
from keras.layers import Layer, InputLayer, Dense, Flatten, Activation, Conv2D, MaxPooling2D, LeakyReLU
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.backend import get_session
from keras.utils.generic_utils import get_custom_objects
import os
from glob import glob
from keras.preprocessing.image import ImageDataGenerator

# Model
The following is the model class which has the methods to do training and prediction. It composes a simple convolution pooling and relu layers stacked to produce a image classification network. The dense/fully connected layer at the end has a custom layer GELU. 

In [21]:
class Model:
    def __init__(self,
                 input_shape=(224, 224, 3),
                 num_classes=2,
                 checkpoint_path="./checkpoint",
                 batch_size=32,
                 epochs=10,
                 learning_rate=0.001):
        """
        input_shape - In HWC format
        """
        self.model = Sequential()
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.checkpoint_path = checkpoint_path
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        get_custom_objects().update({'gelu_activation': Activation(self.gelu_activation)})

    def build_model(self):
        self.model.add(InputLayer(input_shape=self.input_shape))
        self.model.add(Conv2D(32, kernel_size=(3, 3), activation='linear',
                              input_shape=self.input_shape, padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D((2, 2), padding='same'))
        self.model.add(Conv2D(64, (3, 3), activation='linear', padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        self.model.add(
            Conv2D(128, (3, 3), activation='linear', padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation="linear"))
        self.model.add(Activation('gelu_activation', name='GeluActivation'))
        self.model.add(Dense(self.num_classes, activation='softmax'))

#         opt = Adam(learning_rate=self.learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)
        opt = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)


        self.model.compile(optimizer=opt,
                           loss='binary_crossentropy',
                           metrics=['accuracy'])
        print("Model built and compiled successfully")

    @staticmethod
    def gelu_activation(_input, alpha=1):
        return 0.5 * _input * (alpha + tf.tanh(tf.sqrt(2 / pi) * (_input + 0.044715 * _input * _input * _input)))

    def train_model(self, train_data_gen, valid_data_gen):
        checkpoint_dir = os.path.dirname(self.checkpoint_path)
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        tensorboard_dir = os.path.join(checkpoint_dir, 'tensorboard')
        if not os.path.exists(tensorboard_dir):
            os.makedirs(tensorboard_dir)

        # Create a callback that saves the model's weights
        cp_callback = ModelCheckpoint(filepath=self.checkpoint_path, save_weights_only=True, verbose=1, period=1)
        # cp_callback = ModelCheckpoint(filepath=self.checkpoint_path, monitor='val_acc', verbose=1,
        #                               save_best_only=True, mode='max')
        tb_callback = TensorBoard(log_dir=tensorboard_dir, histogram_freq=0, write_graph=True, write_images=False)
        self.model.fit_generator(
            train_data_gen,
            steps_per_epoch=train_data_gen.samples // self.batch_size,
            epochs=self.epochs,
            validation_data=valid_data_gen,
            validation_steps=valid_data_gen.samples // self.batch_size,
            callbacks=[cp_callback, tb_callback])

    def convert_checkpoint(self, final_checkpoint):
        checkpoint_dir = os.path.dirname(final_checkpoint)
        basename = os.path.basename(final_checkpoint).split(".")[0]
        save_path = os.path.join(checkpoint_dir, "tf_ckpt", "final_model.ckpt")
        # Add ops to save and restore all the variables.
        saver = tf.train.Saver()
        self.model.load_weights(final_checkpoint)
        sess = get_session()
        saver.save(sess, save_path)

    def save_frozen(self, frozen_filename):
        # First freeze the graph and remove training nodes.
        output_names = self.model.output.op.name
        sess = get_session()
        frozen_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), [output_names])
        frozen_graph = tf.graph_util.remove_training_nodes(frozen_graph)
        # Save the model
        with open(frozen_filename, "wb") as ofile:
            ofile.write(frozen_graph.SerializeToString())

    def prediction(self, test_data_path):
        from PIL import Image
        import numpy as np
        test_images = glob(os.path.join(test_data_path, "*.jpg"))
        for impath in test_images:
            img = Image.open(impath)
            img = img.resize(self.input_shape[:2])
            img = np.expand_dims(np.array(img), axis=0) / 255.0
            output = self.model.predict(img, batch_size=1)
            print(output)
            pred = np.argmax(output)
            basename = os.path.basename(impath)
            if pred:
                print("{} : Prediction -  Dog".format(basename))
            else:
                print("{} : Prediction -  Cat".format(basename))

# Prepare Dataset
Following is the dataset class to prepare, preprocess and batch the data.

In [22]:
class DataBase:
    def __init__(self, dbpath, input_shape=(224, 224, 3), batch_size=64):
        """

        :param dbpath:  Path to the Dataset. DataBase works on a definite folder structure.
                        Override the functions "fetch_data_paths" and "data_generators" for custom dataset.
        :param input_shape: In format HWC
        """
        self.dbpath = dbpath
        self.input_shape = input_shape
        self.train_dir = os.path.join(self.dbpath, 'train')
        self.valid_dir = os.path.join(self.dbpath, 'valid')
        self.im_height = input_shape[0]
        self.im_width = input_shape[1]
        self.channels = input_shape[2]
        self.batch_size = batch_size

        self.fetch_data_paths()

    def fetch_data_paths(self):
        train_cats_dir = os.path.join(self.train_dir, 'cats')  # directory with our training cat pictures
        train_dogs_dir = os.path.join(self.train_dir, 'dogs')  # directory with our training dog pictures
        validation_cats_dir = os.path.join(self.valid_dir, 'cats')  # directory with our validation cat pictures
        validation_dogs_dir = os.path.join(self.valid_dir, 'dogs')  # directory with our validation dog pictures

        num_cats_tr = len(os.listdir(train_cats_dir))
        num_dogs_tr = len(os.listdir(train_dogs_dir))

        num_cats_val = len(os.listdir(validation_cats_dir))
        num_dogs_val = len(os.listdir(validation_dogs_dir))

        total_train = num_cats_tr + num_dogs_tr
        total_val = num_cats_val + num_dogs_val

        print('total training cat images:', num_cats_tr)
        print('total training dog images:', num_dogs_tr)

        print('total validation cat images:', num_cats_val)
        print('total validation dog images:', num_dogs_val)
        print("--")
        print("Total training images:", total_train)
        print("Total validation images:", total_val)

    def data_generators(self):
        # Prepare Training Data
        train_image_generator = ImageDataGenerator(rotation_range=40,
                                                   width_shift_range=0.2,
                                                   height_shift_range=0.2,
                                                   shear_range=0.2,
                                                   zoom_range=0.2,
                                                   channel_shift_range=10,
                                                   horizontal_flip=True,
                                                   fill_mode='nearest',
                                                   rescale=1. / 255)
        train_batches = train_image_generator.flow_from_directory(self.train_dir,
                                                                  target_size=(self.im_height, self.im_width),
                                                                  interpolation='bicubic',
                                                                  class_mode='categorical',
                                                                  shuffle=True,
                                                                  batch_size=self.batch_size)

        # Prepare Validation Data
        valid_image_generator = ImageDataGenerator(rescale=1. / 255)
        valid_batches = valid_image_generator.flow_from_directory(self.train_dir,
                                                                  target_size=(self.im_height, self.im_width),
                                                                  interpolation='bicubic',
                                                                  class_mode='categorical',
                                                                  shuffle=True,
                                                                  batch_size=self.batch_size)

        return train_batches, valid_batches

# Training 

## Setup training

In [23]:
# Hyper-parameters
batch_size = 128
epochs = 1
learning_rate = 0.001
input_shape = (150, 150, 3)

# Setup Directories
checkpoint_path = "./saved_model/checkpoints/saved_model-{epoch:04d}.h5"
dataset_dir = "/datasets/dogs_cats/train_data/"


In [24]:
# Prepare Dataset
db = DataBase(dataset_dir, input_shape=input_shape, batch_size=batch_size)
train_generator, valid_generator = db.data_generators()

total training cat images: 10000
total training dog images: 10000
total validation cat images: 2500
total validation dog images: 2500
--
Total training images: 20000
Total validation images: 5000
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [25]:
# Build Model
cnn_model = Model(input_shape=input_shape,
                  checkpoint_path=checkpoint_path,
                  epochs=epochs,
                  learning_rate=learning_rate,
                  batch_size=batch_size)
cnn_model.build_model()
cnn_model.model.summary()


Model built and compiled successfully
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
activation_10 (Activation)   (None, 150, 150, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
activation_11 (Activation)   (None, 75, 75, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 38

In [26]:
# Train Model
cnn_model.train_model(train_generator, valid_generator)
cnn_model.prediction("./test_data")











Epoch 1/1
156/156 [==============================] - 322s 2s/step - loss: 0.7019 - acc: 0.5642 - val_loss: 0.6731 - val_acc: 0.5534

Epoch 00001: saving model to ./saved_model/checkpoints/saved_model-0001.h5

[[0.40308756 0.59691244]]
cat.8.jpg : Prediction -  Dog
[[0.3197724  0.68022764]]
cat.7.jpg : Prediction -  Dog
[[0.5552775  0.44472244]]
cat.10.jpg : Prediction -  Cat
[[0.04763709 0.95236295]]
dog.4.jpg : Prediction -  Dog
[[0.3868808  0.61311924]]
dog.3.jpg : Prediction -  Dog
[[0.39767185 0.6023282 ]]
cat.5.jpg : Prediction -  Dog
[[0.35000253 0.64999753]]
cat.6.jpg : Prediction -  Dog
[[0.3325425  0.66745746]]
dog.2.jpg : Prediction -  Dog
[[0.35357457 0.64642537]]
dog.1.jpg : Prediction -  Dog
[[0.363185  0.6368151]]
cat.9.jpg : Prediction -  Dog


In [27]:
# Save Keras Model to Tensorflow Checkpoint
str_epoch = str(epochs).zfill(4)
final_checkpoint = "saved_model/checkpoints/saved_model-{}.h5".format(str_epoch)
cnn_model.convert_checkpoint(final_checkpoint)